In [1]:
import pandas as pd

df = pd.read_csv("../data/fever_small.csv")
df.head()


,claim,label,evidence,id,verifiable,original_id
0,Tie Your Mother Down was released in 2007.,REFUTES,"[['Tie_Your_Mother_Down', '8', ""On several occ...",633922aad8cd96c5e3812afebe56cbc4,VERIFIABLE,209856
1,Matteo Renzi is Italian and was born in Florence.,NOT ENOUGH INFO,"[['Renzi_-LRB-surname-RRB-', '22', 'Matteo Ren...",612be5782034160ae0b7d1c21539d740,NOT VERIFIABLE,172464
2,Ashley Graham was never on a magazine cover.,REFUTES,"[['Ashley_Graham_-LRB-model-RRB-', '6', 'In 20...",65cc3f98bece69f9d3a724df9cac82d6,VERIFIABLE,126982
3,Luis Fonsi is Puerto Rican.,SUPPORTS,"[['Luis_Fonsi', '0', 'Luis Alfonso Rodríguez L...",1aa8074ed9e9c0e067ffe4e8549ba980,VERIFIABLE,75871
4,Murda Beatz is a vegan.,NOT ENOUGH INFO,"[['Coke_N_Butter', '2', 'The project consists ...",77fd25cfc2a6099095b27aeb11a6d086,NOT VERIFIABLE,135082


In [4]:
from sentence_transformers import SentenceTransformer

embedder = SentenceTransformer("all-MiniLM-L6-v2")

docs = df["evidence"].astype(str).tolist()

embeddings = embedder.encode(docs, show_progress_bar=True)


Loading weights:   0%|          | 0/103 [00:00<?, ?it/s]

BertModel LOAD REPORT from: sentence-transformers/all-MiniLM-L6-v2
Key                     | Status     |  | 
------------------------+------------+--+-
embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


Batches:   0%|          | 0/16 [00:00<?, ?it/s]

In [5]:
import faiss
import numpy as np

dim = embeddings.shape[1]

index = faiss.IndexFlatL2(dim)
index.add(np.array(embeddings))

print("FAISS index built with", index.ntotal, "documents ✅")


FAISS index built with 500 documents ✅


In [6]:
faiss.write_index(index, "../retrieval/fever.index")

print("Index saved to retrieval/fever.index 💾")


Index saved to retrieval/fever.index 💾


In [7]:
import pandas as pd

df = pd.read_csv("../data/fever_small.csv")

print("Loaded dataframe with", len(df), "rows ✅")
df.head()


Loaded dataframe with 500 rows ✅


,claim,label,evidence,id,verifiable,original_id
0,Tie Your Mother Down was released in 2007.,REFUTES,"[['Tie_Your_Mother_Down', '8', ""On several occ...",633922aad8cd96c5e3812afebe56cbc4,VERIFIABLE,209856
1,Matteo Renzi is Italian and was born in Florence.,NOT ENOUGH INFO,"[['Renzi_-LRB-surname-RRB-', '22', 'Matteo Ren...",612be5782034160ae0b7d1c21539d740,NOT VERIFIABLE,172464
2,Ashley Graham was never on a magazine cover.,REFUTES,"[['Ashley_Graham_-LRB-model-RRB-', '6', 'In 20...",65cc3f98bece69f9d3a724df9cac82d6,VERIFIABLE,126982
3,Luis Fonsi is Puerto Rican.,SUPPORTS,"[['Luis_Fonsi', '0', 'Luis Alfonso Rodríguez L...",1aa8074ed9e9c0e067ffe4e8549ba980,VERIFIABLE,75871
4,Murda Beatz is a vegan.,NOT ENOUGH INFO,"[['Coke_N_Butter', '2', 'The project consists ...",77fd25cfc2a6099095b27aeb11a6d086,NOT VERIFIABLE,135082


In [8]:
docs = df["evidence"].astype(str).tolist()

answers = []

for claim in df["claim"][:50]:
    idxs = retrieve_docs(claim)
    context = " ".join([docs[i] for i in idxs])

    prompt = f"""
Use ONLY the text below to answer the claim.

TEXT:
{context}

CLAIM:
{claim}

Answer whether the claim is true or false and explain briefly.
"""

    out = generator(prompt, max_new_tokens=120)
    answers.append(out[0]["generated_text"])

print("Generated", len(answers), "answers ✅")


NameError: name 'retrieve_docs' is not defined

In [9]:
def retrieve_docs(query, k=3):
    q_emb = embedder.encode([query])
    distances, indices = index.search(np.array(q_emb), k)
    return indices[0]


In [10]:
import pandas as pd
df = pd.read_csv("../data/fever_small.csv")


In [11]:
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np

embedder = SentenceTransformer("all-MiniLM-L6-v2")
index = faiss.read_index("../retrieval/fever.index")


Loading weights:   0%|          | 0/103 [00:00<?, ?it/s]

BertModel LOAD REPORT from: sentence-transformers/all-MiniLM-L6-v2
Key                     | Status     |  | 
------------------------+------------+--+-
embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


In [12]:
def retrieve_docs(query, k=3):
    q_emb = embedder.encode([query])
    distances, indices = index.search(np.array(q_emb), k)
    return indices[0]


In [13]:
docs = df["evidence"].astype(str).tolist()

answers = []

for claim in df["claim"][:50]:  # first 50 only
    idxs = retrieve_docs(claim)
    context = " ".join([docs[i] for i in idxs])

    prompt = f"""
Use ONLY the text below to answer the claim.

TEXT:
{context}

CLAIM:
{claim}

Answer whether the claim is true or false and explain briefly.
"""

    out = generator(prompt, max_new_tokens=120)
    answers.append(out[0]["generated_text"])

print("Generated", len(answers), "answers ✅")


NameError: name 'generator' is not defined

In [14]:
from transformers import pipeline

generator = pipeline(
    "text-generation",
    model="TinyLlama/TinyLlama-1.1B-Chat-v1.0",
    device_map="auto"
)

print("Generator loaded ✅")


Loading weights:   0%|          | 0/201 [00:00<?, ?it/s]

Generator loaded ✅


In [15]:
docs = df["evidence"].astype(str).tolist()

answers = []

for claim in df["claim"][:50]:  # first 50 only
    idxs = retrieve_docs(claim)
    context = " ".join([docs[i] for i in idxs])

    prompt = f"""
Use ONLY the text below to answer the claim.

TEXT:
{context}

CLAIM:
{claim}

Answer whether the claim is true or false and explain briefly.
"""

    out = generator(prompt, max_new_tokens=120)
    answers.append(out[0]["generated_text"])

print("Generated", len(answers), "answers ✅")


Passing `generation_config` together with generation-related arguments=({'max_new_tokens'}) is deprecated and will be removed in future versions. Please pass either a `generation_config` object OR all generation parameters explicitly, but not both.
Both `max_new_tokens` (=120) and `max_length`(=2048) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both `max_new_tokens` (=120) and `max_length`(=2048) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both `max_new_tokens` (=120) and `max_length`(=2048) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation

Generated 50 answers ✅


In [ ]:
docs = df["evidence"].astype(str).tolist()

answers = []

for claim in df["claim"][:50]:  # first 50 only
    idxs = retrieve_docs(claim)
    context = " ".join([docs[i] for i in idxs])

    prompt = f"""
Use ONLY the text below to answer the claim.

TEXT:
{context}

CLAIM:
{claim}

Answer whether the claim is true or false and explain briefly.
"""

    out = generator(prompt, max_new_tokens=120)
    answers.append(out[0]["generated_text"])

print("Generated", len(answers), "answers ✅")


Passing `generation_config` together with generation-related arguments=({'max_new_tokens'}) is deprecated and will be removed in future versions. Please pass either a `generation_config` object OR all generation parameters explicitly, but not both.
Both `max_new_tokens` (=120) and `max_length`(=2048) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both `max_new_tokens` (=120) and `max_length`(=2048) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both `max_new_tokens` (=120) and `max_length`(=2048) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation

In [ ]:
docs = df["evidence"].astype(str).tolist()

answers = []

for claim in df["claim"][:50]:  # first 50 only
    idxs = retrieve_docs(claim)
    context = " ".join([docs[i] for i in idxs])

    prompt = f"""
Use ONLY the text below to answer the claim.

TEXT:
{context}

CLAIM:
{claim}

Answer whether the claim is true or false and explain briefly.
"""

    out = generator(prompt, max_new_tokens=120)
    answers.append(out[0]["generated_text"])

print("Generated", len(answers), "answers ✅")


Passing `generation_config` together with generation-related arguments=({'max_new_tokens'}) is deprecated and will be removed in future versions. Please pass either a `generation_config` object OR all generation parameters explicitly, but not both.
Both `max_new_tokens` (=120) and `max_length`(=2048) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both `max_new_tokens` (=120) and `max_length`(=2048) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both `max_new_tokens` (=120) and `max_length`(=2048) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation

In [16]:
docs = df["evidence"].astype(str).tolist()

answers = []

for claim in df["claim"][:10]:
  # first 10 only
    idxs = retrieve_docs(claim)
    context = " ".join([docs[i] for i in idxs])

    prompt = f"""
Use ONLY the text below to answer the claim.

TEXT:
{context}

CLAIM:
{claim}

Answer whether the claim is true or false and explain briefly.
"""

    out = generator(prompt, max_new_tokens=120)
    answers.append(out[0]["generated_text"])

print("Generated", len(answers), "answers ✅")


Both `max_new_tokens` (=120) and `max_length`(=2048) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both `max_new_tokens` (=120) and `max_length`(=2048) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both `max_new_tokens` (=120) and `max_length`(=2048) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both `max_new_tokens` (=120) and `max_length`(=2048) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generati

Generated 10 answers ✅
